In [ ]:
import pygame
import numpy as np
import math
import random

pygame.init()
WIDTH, HEIGHT = 800, 800
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("旋轉正方形內的彈跳球")
clock = pygame.time.Clock()

# 顏色定義
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
BALL_COLOR = (255, 100, 100)
SQUARE_COLOR = (100, 255, 100)

# 模擬參數
GRAVITY = np.array([0, 0.5])
NUM_BALLS = 5
BALL_RADIUS = 15
BOUNCE = 0.9

# 正方形邊界參數
SQUARE_SIZE = 400
ROTATION_SPEED = 0.01  # 每幀旋轉角度（弧度）

# 球類別
class Ball:
    def __init__(self):
        self.pos = np.array([
            WIDTH / 2 + random.uniform(-100, 100),
            HEIGHT / 2 + random.uniform(-100, 100)
        ], dtype=float)
        self.vel = np.array([random.uniform(-2, 2), random.uniform(-2, 2)], dtype=float)

    def update(self):
        self.vel += GRAVITY
        self.pos += self.vel

    def draw(self, surface):
        pygame.draw.circle(surface, BALL_COLOR, self.pos.astype(int), BALL_RADIUS)

# 建立球
balls = [Ball() for _ in range(NUM_BALLS)]

# 畫出旋轉的正方形並回傳頂點位置
def draw_rotating_square(surface, angle):
    center = np.array([WIDTH / 2, HEIGHT / 2])
    half = SQUARE_SIZE / 2
    # 初始四個頂點（未旋轉）
    corners = [
        np.array([-half, -half]),
        np.array([ half, -half]),
        np.array([ half,  half]),
        np.array([-half,  half])
    ]
    # 建立旋轉矩陣
    rotation_matrix = np.array([
        [math.cos(angle), -math.sin(angle)],
        [math.sin(angle),  math.cos(angle)]
    ])
    # 旋轉並平移頂點
    transformed = [center + rotation_matrix @ c for c in corners]

    # 畫邊界
    for i in range(4):
        pygame.draw.line(surface, SQUARE_COLOR, transformed[i], transformed[(i+1)%4], 4)
    return transformed

# 計算球到邊的最近距離 + 反彈
def reflect_if_hit(ball, square_corners):
    for i in range(4):
        p1 = square_corners[i]
        p2 = square_corners[(i + 1) % 4]
        edge = p2 - p1
        to_ball = ball.pos - p1
        proj = np.dot(to_ball, edge) / np.dot(edge, edge)
        proj = np.clip(proj, 0, 1)
        closest = p1 + proj * edge
        dist = np.linalg.norm(ball.pos - closest)
        if dist < BALL_RADIUS:
            normal = ball.pos - closest
            normal = normal / np.linalg.norm(normal)
            ball.pos = closest + normal * BALL_RADIUS  # 推出邊界
            v_dot_n = np.dot(ball.vel, normal)
            ball.vel = ball.vel - (1 + BOUNCE) * v_dot_n * normal

# 主迴圈
running = True
angle = 0
while running:
    screen.fill(BLACK)
    dt = clock.tick(60) / 1000.0

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # 畫旋轉正方形並取得邊界
    corners = draw_rotating_square(screen, angle)
    angle += ROTATION_SPEED

    # 更新每顆球
    for ball in balls:
        ball.update()
        reflect_if_hit(ball, corners)
        ball.draw(screen)

    pygame.display.flip()

pygame.quit()


pygame 2.6.0 (SDL 2.28.4, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


2025-04-17 18:55:56.265 python[5699:41403] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit


: 